# Desafio Coodesh - Análise e Processamento dos Dados
Análise e transformação dos dados utilizando Python e Pandas

In [1]:
from datetime import datetime
import pandas as pd

## Atributos dos datasets

In [2]:
dfAtributos = pd.read_csv("./analytics/dataset_info.csv")
dfAtributos

,seq,name,description
0,1,show_id,Unique ID for every Movie / Tv Show
1,2,type,Identifier - A Movie or TV Show
2,3,title,Title of the Movie / Tv Show
3,4,director,Director of the Movie
4,5,cast,Actors involved in the movie / show
5,6,country,Country where the movie / show was produced
6,7,data_added,Date it was added on Netflix or Amazon Prime
7,8,release_year,Actual Release year of the move / show
8,9,rating,TV Rating of the movie / show
9,10,duration,Total Duration - in minutes or number of seasons


(*) Ao comparar o arquivo attributes.txt com o dataset de filmes, foi possível identificar que faltavam colunas no arquivo de atributos. Foi feita a identificação dos campos e atualizada a documentação.

## Carregar os arquivos

In [3]:
dfFilmesNetflix = pd.read_csv("./data/netflix_titles.csv")
dfFilmesNetflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [4]:
dfFilmesAmazon = pd.read_csv("./data/amazon_prime_titles.csv")
dfFilmesAmazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


# Gerar um dataset unificado de filmes das duas plataformas

In [5]:
# Como o campo show_id se repete em cada plataforma, 
# será necessário garantir que exista um código identificador único para cada dataset
# Para não perder a referência da plataforma, é necessário criar uma coluna com esta informação em cada dataframe

dfFilmesNetflix["title_id"] = "nfx_" + dfFilmesNetflix["show_id"]
dfFilmesNetflix["platform"] = "netflix"

dfFilmesAmazon["title_id"] = "amz_" + dfFilmesAmazon["show_id"]
dfFilmesAmazon["platform"] = "amazon_prime" 

dfFilmesNetflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
 12  title_id      8807 non-null   object
 13  platform      8807 non-null   object
dtypes: int64(1), object(13)
memory usage: 963.4+ KB


In [6]:
dfFilmesAmazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
 12  title_id      9668 non-null   object
 13  platform      9668 non-null   object
dtypes: int64(1), object(13)
memory usage: 1.0+ MB


### Criar base de filmes unificada, contendo os filmes das duas plataformas

In [7]:
dfFilmes = pd.concat([dfFilmesNetflix, dfFilmesAmazon],ignore_index=True)
dfFilmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18475 entries, 0 to 18474
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       18475 non-null  object
 1   type          18475 non-null  object
 2   title         18475 non-null  object
 3   director      13758 non-null  object
 4   cast          16417 non-null  object
 5   country       8648 non-null   object
 6   date_added    8952 non-null   object
 7   release_year  18475 non-null  int64 
 8   rating        18134 non-null  object
 9   duration      18472 non-null  object
 10  listed_in     18475 non-null  object
 11  description   18475 non-null  object
 12  title_id      18475 non-null  object
 13  platform      18475 non-null  object
dtypes: int64(1), object(13)
memory usage: 2.0+ MB


## Fazer o processamento do dataset unificado

Para modelagem dos dados, é necessário transformar as colunas que o conteúdo representa uma lista, em um arquivos de dados que conterá o relacionamento com o identificador do título.

* director -> directors_title.csv
* cast -> casting_title.csv
* listed_in -> categories_title.csv
* country -> countries_title.csv

Para facilitar o trabalho com data, o campo date_added precisa ser convertido para o formato YYYY-MM-DD.<br>
Uma vez que as colunas com conteúdo de lista foram extraídas, as mesmas não precisam compor o dataset de títulos.

### Avaliação de títulos duplicados

In [8]:
dfFilmes[dfFilmes['title_id'].duplicated() == True].info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       0 non-null      object
 1   type          0 non-null      object
 2   title         0 non-null      object
 3   director      0 non-null      object
 4   cast          0 non-null      object
 5   country       0 non-null      object
 6   date_added    0 non-null      object
 7   release_year  0 non-null      int64 
 8   rating        0 non-null      object
 9   duration      0 non-null      object
 10  listed_in     0 non-null      object
 11  description   0 non-null      object
 12  title_id      0 non-null      object
 13  platform      0 non-null      object
dtypes: int64(1), object(13)
memory usage: 0.0+ bytes


In [9]:
# Função para extrair os valores dos campos que apresentam lista e relacionar ao id do título
def extrairListaDeRelacionamento(df:pd.DataFrame, coluna:str, camposExtras=[]):
    lista = []
    for index,linha in df.iterrows():
        if not pd.isna(linha[coluna]):
            id = linha["title_id"]
            itens = str(linha[coluna]).split(",")
            for item in itens:
                lista.append([id,item.strip()])
    return lista

In [10]:
# Criar dataset contendo os diretores e os filmes dirigidos por eles
listaDiretores = extrairListaDeRelacionamento(dfFilmes,"director")
dfDirector = pd.DataFrame(listaDiretores,columns = ['title_id', 'director'])
dfDirector.to_csv("./analytics/directors_title.csv",index=False)

dfDirector.describe()

,title_id,director
count,15212,15212
unique,13758,10488
top,amz_s4964,Mark Knight
freq,17,114


In [11]:
# Criar dataset contendo os artistas do elenco do filme
listaArtistas = extrairListaDeRelacionamento(dfFilmes,"cast")
dfCasting = pd.DataFrame(listaArtistas,columns = ['title_id', 'artist'])
dfCasting.to_csv("./analytics/casting_title.csv",index=False)

dfCasting.describe()

,title_id,artist
count,108493,108493
unique,16417,60761
top,amz_s4219,Anupam Kher
freq,76,59


In [12]:
# Criar o dataset de países relacionados ao filme
listaPaises = extrairListaDeRelacionamento(dfFilmes,"country")
dfCountries = pd.DataFrame(listaPaises,columns = ['title_id', 'country'])
# Incluir no dataset dos países a plataforma
dfCountries = dfCountries.merge(
    dfFilmes.rename({'country': 'country_list'}, axis=1),
    on='title_id',
    how='left')[["title_id","country","platform"]]

dfCountries.to_csv("./analytics/countries_title.csv",index=False)
dfCountries.describe()

,title_id,country,platform
count,10843,10843,10843
unique,8648,125,2
top,nfx_s6234,United States,netflix
freq,12,4024,10019


In [13]:
# Criar dataset das categorias as quais um filme foi classificado
listaCategorias = extrairListaDeRelacionamento(dfFilmes,"listed_in")
dfCategories = pd.DataFrame(listaCategorias,columns = ['title_id', 'category'])
dfCategories.to_csv("./analytics/categories_title.csv",index=False)

dfCategories.describe()

,title_id,category
count,37632,37632
unique,18475,72
top,amz_s3619,Drama
freq,5,3687


In [14]:
# Função para converter a data de inclusão na plataforma para o formato ISO.
# A função é necessário pois a função nativa do pandas.to_datetime não consegue converter a data no formato original pelo fato do dia ser "zero-padding"
def converterData(data:str):
    try:
        mesDia,ano = data.split(", ")
        mes,dia = mesDia.split(" ")
        dataTexto =  f"{mes} {dia.zfill(2)}, {ano}"
        novaData = datetime.strptime(dataTexto,"%B %d, %Y")
        return novaData.strftime("%Y-%m-%d")
    except Exception as e:
        pass

In [15]:
# Converter a data de inclusão na plataforma paratodos os registros do dataset
dfFilmes["date_added"] = dfFilmes["date_added"].apply(converterData)
dfFilmes.describe()

,release_year
count,18475.000000
mean,2011.124980
std,15.262484
min,1920.000000
25%,2011.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


In [16]:
#Excluir colunas de lista que foram convertidas em datasets de relacionamento
novoDfFilmes = dfFilmes.drop(columns=["director","cast","country","listed_in"])

#Salvar novo dataset de filmes unificado
novoDfFilmes.to_csv("./analytics/titles.csv",index=False)